In [1]:
%pip install --quiet openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


### Deleting Contents in the JSON file

In [49]:
import json

def clear_sentiment_results(file_name):
    file_path = "results/" + file_name
    try:
        # Open the file in write mode, which will clear its contents
        with open(file_path, 'w') as f:
            # Write an empty list to the file
            json.dump([], f)
        print(f"All objects removed from {file_path}. The file now contains an empty list.")
    except Exception as e:
        print(f"An error occurred while clearing the file: {str(e)}")

# Call the function to clear the file
clear_sentiment_results("sentiment_results.json")

All objects removed from results/sentiment_results.json. The file now contains an empty list.


### New Code with tracking of reasonings and updating prompt

In [14]:
import json

def store_result(result, file_name):
    file_path = "results/" + file_name
    try:
        # Read existing results
        with open(file_path, 'r') as f:
            results = json.load(f)
    except FileNotFoundError:
        # If file doesn't exist, start with an empty list
        results = []
    
    # Append new result
    results.append(result)
    
    # Write updated results back to file
    with open(file_path, 'w') as f:
        json.dump(results, f, indent=2)

In [53]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Access the API key
api_key = os.getenv('OPENAI_API_KEY')

# Create an OpenAI client
client = OpenAI(api_key=api_key)
    
import json
from openai import OpenAI

client = OpenAI()


sentiment_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "sentiment_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "boolean",
                    "description": "True if in favor of the proposition, false if against"
                },
                "core_reason": {
                    "type": "string",
                    "description": "Brief core argument in sentiment analysis"
                },
                "nuance": {
                    "type": "string",
                    "description": "A new nuance for the core reason, if applicable; empty string if not"
                }
            },
            "required": ["sentiment", "core_reason","nuance"],
            "additionalProperties": False
        }
    }
}

def analyze_opinion(question, opinion, previous_results):
    try:
        # Create a string of previous results with their counts
        previous_results_str = ", ".join([f"{reason} ({data['count']} times)" for reason, data in previous_results.items()])

        system_message = f"""You are a helpful assistant analyzing opinions on the question: '{question}'.
        Your task is to categorize the given opinion into an appropriate core reason.

        Guidelines:
        1. Determine the sentiment (true if the opinion is answering yes to the question, false if answering no).
        2. Identify a brief core reason ( 5-7 words) that captures the essence of the opinion.
        3. IMPORTANT: Carefully consider if the opinion fits into an existing core reason. Only create a new core reason if the opinion presents a substantially different argument.
        4. If the core reason is similar to an existing one, use the EXACT same wording.

        Previous core reasons and their frequencies:
        {previous_results_str}

        While there's no strict limit, aim to group similar reasons together when possible to maintain a manageable set of distinct core reasons.

        Provide your response in the required JSON format, including sentiment and core_reason. Leave the nuance field empty for now.

        Opinion: {opinion}
        """

        # Initial call for sentiment and core reason
        initial_response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": opinion}
            ],
            response_format=sentiment_schema,
            temperature=0.2
        )

        initial_result = json.loads(initial_response.choices[0].message.content)
        core_reason = initial_result['core_reason']
        existing_nuances = previous_results.get(core_reason, {}).get('nuances', [])

        nuance_message = f"""Now that you've identified the core reason as "{core_reason}", 
        analyze if a new nuance is needed.

        Original opinion: {opinion}

        Instructions:
        1. Carefully analyze the original opinion and compare it to the existing nuances.

        Here are the existing nuances for this reason, if any:
        {json.dumps(existing_nuances, indent=2)}

        2. Determine if the opinion presents a SIGNIFICANTLY different perspective or important detail not covered by existing nuances.

        3. Output:
           - If the opinion offers a significant new perspective, provide a concise new nuance (max 15 words).
           - If the opinion doesn't add significant new information, output an empty string.

        Provide your response as a simple string (the new nuance or an empty string).
        """

        nuance_response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18", 
            messages=[
                {"role": "system", "content": nuance_message},
                {"role": "user", "content": opinion}
            ],
            temperature=0.2,
            max_tokens=150
        )

        new_nuance = nuance_response.choices[0].message.content.strip()
        # Add the new nuance to the initial result
        initial_result['nuance'] = new_nuance

        return initial_result

    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {str(e)}")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [15]:
from difflib import SequenceMatcher

def is_similar(a, b, threshold=0.7):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() > threshold

In [50]:
def update_core_reasons(result, previous_results):
    try:
        core_reason = result['core_reason']
        sentiment = result['sentiment']
        new_nuance = result['nuance']
        
        if core_reason not in previous_results:
            previous_results[core_reason] = {
                'sentiment': sentiment,
                'nuances': [],
                'count': 1
            }
        else:
            previous_results[core_reason]['count'] += 1
        
        if new_nuance:
            previous_results[core_reason]['nuances'].append(new_nuance)
        
        return previous_results
    except Exception as e:
        print(f"An error occurred while updating core reasons: {str(e)}")
        return previous_results

In [55]:
if __name__ == "__main__":
    file_name = "sentiment_results.json"
    question = "Should phones be banned from schools?"
    
    
    opinions = [
    "Just as more kids began spending more time with their phones, we saw a massive spike in depression and mental illness.",
    "Phones prevent socialization between students during school.",
    "Despite what rules may exist, most students are using their phones during school",
    "Phone usage reduces learning",
    "Having a smartphone with you at all times gives you the ability to instantly communicate with someone else. Students are able to contact parents, guardians or the authorities without much hassle and vice versa.",
    "Smartphones are all about the apps and the amazing things they can do, these apps can be used in a number of creative ways to facilitate their classroom learning experience.",
    "With phones, students can access research, news and videos to enhance their learning.",
    "Smartphones can be utilized for digital harassment in and out of school.",
    "Students could become highly distracted from the many sources of entertainment.",
    "Frequent usage of smartphones has been linked to negative effects on both physical and mental health.",
    "Banning phones would eliminate the problem of cyberbullying during school hours.",
    "The presence of phones in school creates inequality between students who can and cannot afford them.",
    "Phones are essential for students with certain medical conditions to monitor their health.",
    "The use of phones in schools prepares students for the technology-driven workforce they'll enter.",
    "Phones can be used to cheat on tests and assignments, compromising academic integrity.",
    "Banning phones would make it harder for students to coordinate after-school activities and rides home.",
    "Phones can be used to document bullying or other inappropriate behavior in schools.",
    "Allowing phones in school teaches students responsible use of technology.",
    "Banning phones would reduce the risk of theft and property damage in schools.",
    "The radiation from multiple phones in a classroom could potentially be harmful to health.",
    "Phones can be disruptive when they ring or vibrate during class.",
    "Banning phones would make it harder for students to balance part-time jobs and school responsibilities.",
    "Phones can be used to take photos of notes and assignments, helping students stay organized.",
    "Allowing phones in school helps bridge the digital divide for students without internet access at home.",
    "The use of phones in school undermines the authority of teachers and school administration."
    ]
    '''
    opinions = [
        "Students need phones for academic work.",
        "Using phones help students because a lot of them dont have calculators so they use a calculator app",
        "Phones also allow students to offer video recordings of their work in class"

    ]
    '''    
    
    previous_results = {}

    for opinion in opinions:
        result = analyze_opinion(question, opinion, previous_results)
        if result:
            core_reason = result['core_reason']
            if core_reason not in previous_results:
                previous_results[core_reason] = {
                    'sentiment': result['sentiment'],
                    'nuances': [],
                    'count': 1
                }
            else:
                previous_results[core_reason]['count'] += 1
            
            if 'nuance' in result and result['nuance']:  # Only add if nuance exists and is non-empty
                previous_results[core_reason]['nuances'].append(result['nuance'])
            
            store_result(result, file_name)

    # Print out the core reasons, sentiments, and nuances
    print("\nCore Reasons, Sentiments, and Nuances:")
    for core_reason, data in previous_results.items():
        sentiment = "pro" if data['sentiment'] else "con"
        print(f"\nCore Reason: {core_reason}")
        print(f"Sentiment: {sentiment}")
        print(f"Mentioned {data['count']} time(s)")
        print("Nuances:")
        
        # Filter out empty strings and remove quotes
        non_empty_nuances = [nuance.strip("\"'") for nuance in data['nuances'] if nuance.strip()]
        
        if non_empty_nuances:
            for nuance in non_empty_nuances:
                print(f"- {nuance}")
        else:
            print("- No specific nuances recorded.")
        print("-" * 50)


Core Reasons, Sentiments, and Nuances:

Core Reason: Increased mental health issues among students
Sentiment: pro
Mentioned 5 time(s)
Nuances:
- Increased screen time correlates with rising anxiety and social isolation among students.
- Digital harassment via smartphones exacerbates mental health issues among affected students.
- 
- Banning phones could directly reduce opportunities for cyberbullying in schools.
- 
--------------------------------------------------

Core Reason: Phones prevent socialization between students
Sentiment: pro
Mentioned 7 time(s)
Nuances:
- Phones create barriers to face-to-face interactions among students in educational settings.
- Phones distract students from engaging with peers and participating in classroom activities.
- 
- 
- 
- 
- 
--------------------------------------------------

Core Reason: Enhances learning with access to information
Sentiment: con
Mentioned 5 time(s)
Nuances:
- 
- Prepares students for future careers by integrating technology